# Set Up Matrix Input

In [1]:
import numpy as np
import time

# Row and Col size of matrix being bulitiplied
row_size = 4
col_size = 64

# Generate random input matrix
A = np.random.rand(row_size,col_size)
B = np.random.rand(col_size,row_size)
# A = np.full((row_size,col_size),1/(2**5))
# B = np.full((col_size,row_size),1/(2**5))
# A = np.random.uniform(low=0.0625,high=1,size=(row_size,col_size))
# B = np.random.uniform(low=0.0625,high=1,size=(col_size,row_size))

# pos_values = [2**(-i) for i in range(0,2)]
# neg_values = [-2**(-i) for i in range(0,2)]
# values = np.concatenate([neg_values,pos_values,[0]])
# A_val = np.random.choice(values,row_size*col_size)
# A = A_val.reshape(row_size,col_size)
# B_val = np.random.choice(values,row_size*col_size)
# B = B_val.reshape(col_size,row_size)

# A = np.zeros((row_size,col_size))
# A[::2, :] = 1
# A[1::2,:] = -0.5
# B = np.zeros((col_size,row_size))
# B[::2, :] = -0.5
# B[1::2,:] = 0.25

py_out = np.zeros((row_size,row_size))
py_out = np.matmul(A,B)

print(f"{A}\n{B}\n{py_out}")

[[0.39692388 0.84154197 0.20277561 0.39031878 0.78906842 0.34569173
  0.97681999 0.45422132 0.08131032 0.09763499 0.5133081  0.69440149
  0.96559592 0.98724502 0.75223214 0.62252877 0.77421919 0.06931729
  0.35698736 0.67807893 0.31893628 0.85542541 0.04276872 0.36026313
  0.22517422 0.15778599 0.52177154 0.97822427 0.81844098 0.51154195
  0.99038701 0.074255   0.5769376  0.2850126  0.78491679 0.31385363
  0.34135336 0.04716045 0.49572093 0.272058   0.20576347 0.42236706
  0.89890433 0.98454036 0.99927919 0.29334356 0.89037096 0.77415796
  0.64690371 0.86672583 0.57251821 0.33451368 0.72336813 0.36388352
  0.99291711 0.62420404 0.16881416 0.98226814 0.74788996 0.77162507
  0.29454077 0.88742449 0.59731463 0.28434457]
 [0.75016663 0.65574173 0.66071574 0.97024369 0.15188547 0.42513741
  0.5633688  0.4313325  0.99920098 0.13214849 0.96069047 0.0279097
  0.17573209 0.61709341 0.17920711 0.31790632 0.14095165 0.54666425
  0.41933816 0.82056011 0.84108636 0.76201286 0.06818382 0.97773364
  

# Converters Float to Fixed Point and Vice Versa

In [2]:
def float_to_fp(value, total_bits=16, frac_bits=8, output_format='hex'):
    # Step 1: Check if the value is negative
    if value < 0:
        is_negative = True
        value = -value  # Work with the positive equivalent for conversion
    else:
        is_negative = False

    # Step 2: Multiply the float by 2^frac_bits to scale it
    scaled_value = round(value * (2 ** frac_bits))
    
    # Step 3: Convert the scaled value to a binary string
    # Format the binary string to the required total bits length
    binary_value = bin(scaled_value)[2:].zfill(total_bits)

    # Step 4: Handle the negative case
    if is_negative:
        # Perform two's complement conversion for negative numbers
        # Subtract the binary value from the max value for two's complement
        max_value = (1 << total_bits)  # This is 2^total_bits
        binary_value = bin(max_value - scaled_value)[2:].zfill(total_bits)

    # Step 5: Convert the binary value to hexadecimal
    hex_value = hex(int(binary_value, 2))[2:].zfill(total_bits // 4)  # Convert to hex and remove '0x'

    # Step 6: Format output based on the `output_format` argument
    if output_format == 'binary':
        return binary_value
    elif output_format == 'hex':
        return hex_value
    elif output_format == 'both':
        return (f"{total_bits}'b{binary_value}", f"{total_bits}'h{hex_value}")
    else:
        raise ValueError("Invalid output format. Choose 'binary', 'hex', or 'both'.")
    
def fp_to_float(value,total_bits=16, frac_bits=8, input_format='hex'):
    if input_format == 'binary': fp_val = int(value,2)
    elif input_format == 'hex': fp_val = int(value,16)
    else: raise ValueError("Invalid input format. Must be 'binary' or 'hex'")
    # Calculate the scaling factor: 2^frac_width
    scaling_factor = 2 ** frac_bits
    # Handle two's complement if the sign bit is set (for negative numbers)
    sign_bit_mask = 1 << (total_bits - 1)  # Mask to isolate the sign bit
    
    # Check if the value is negative by examining the sign bit
    if fp_val & sign_bit_mask:  # If sign bit is 1, it's negative
        # Convert from two's complement
        fp_val = fp_val - (1 << total_bits)  # Apply the two's complement adjustment
    
    # Convert to float by dividing by the scaling factor
    float_value = fp_val / scaling_factor
    
    return float_value

# Example usage:
value = 64  # Example float number

num1 = float_to_fp(value)
inv_num1 = fp_to_float(str(num1))
print(num1,inv_num1)

4000 64.0


# Create File for RTL Testing

In [3]:
def wr_input(matrix,filename):
    cnt = 0
    f = open(filename,'w')
    for i in range(8):
        for j in range(15):
            if j == 0:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            elif j == 1:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            elif j == 2:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(0)
            elif j == 8:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            elif j == 9:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            elif j == 10:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(matrix[i][3])
            elif j > 10:
                elem0 = float_to_fp(0)
                elem1 = float_to_fp(0)
                elem2 = float_to_fp(0)
                elem3 = float_to_fp(0)
            else:
                elem0 = float_to_fp(matrix[i][0])
                elem1 = float_to_fp(matrix[i][1])
                elem2 = float_to_fp(matrix[i][2])
                elem3 = float_to_fp(matrix[i][3])
            
            f.write(f'{elem0}{elem1}{elem2}{elem3}\n')
    f.close()

wr_input(A.transpose(),"..\\data_west.txt")
wr_input(B,"..\\data_north.txt")

# Run Testbench

In [4]:
%%cmd
cd ..
iverilog.exe -c .\block_4x32.txt
vvp .\a.out

Microsoft Windows [Version 10.0.22631.4602]
(c) Microsoft Corporation. All rights reserved.

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication\python>cd ..

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication>iverilog.exe -c .\block_4x32.txt

c:\Users\fauza\Documents\Univ\Sem_7\EL5035_VLSI_for_Digital_System_Processing\project\block_based_matrix_multiplication>vvp .\a.out
VCD info: dumpfile wave.vcd opened for output.
VCD warning: array word block_4x32_tb.data_north[0] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_west[0] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_north[1] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_west[1] will conflict with an escaped identifier.
VCD warning: array word block_4x32_tb.data_north[2] will conflict 

# Read RTL Output File and Check Error

In [5]:
rtl_out = np.zeros((row_size,row_size))

with open('..\\out_val.txt', 'r') as f:
    # Iterate through each line in the file
    for i, line in enumerate(f):
        # Strip newline characters or extra spaces
        line = line.strip()
        
        # Parse the line into chunks of 4 characters
        for j in range(row_size):
            start_index = j * 4
            end_index = start_index + 4
            temp = line[start_index:end_index]
            rtl_out[i][j] = fp_to_float(temp)


err = py_out - rtl_out
# print(f"{py_out}\n{rtl_out}\n{err}")
print(err)

0d40
17c8
12b0
1348
0b48
1610
0e58
1478
0998
1250
0e88
10f0
0828
0fe0
0ba8
0da8
[[16.190767   16.98296909 18.88994751 19.8601479 ]
 [14.72057197 13.19592083 14.13531625 17.3310177 ]
 [16.41948066 16.69998813 16.88939145 20.7962616 ]
 [14.17329912 13.87654089 14.58273037 16.03535025]]
[[13.25    23.78125 18.6875  19.28125]
 [11.28125 22.0625  14.34375 20.46875]
 [ 9.59375 18.3125  14.53125 16.9375 ]
 [ 8.15625 15.875   11.65625 13.65625]]
[[ 2.940767   -6.79828091  0.20244751  0.5788979 ]
 [ 3.43932197 -8.86657917 -0.20843375 -3.1377323 ]
 [ 6.82573066 -1.61251187  2.35814145  3.8587616 ]
 [ 6.01704912 -1.99845911  2.92648037  2.37910025]]
